### convert hotpotqa to squard format

According to Longformer: use the following input format with special tokens:  “[CLS] [q] question [/q] [p] sent1,1 [s] sent1,2 [s] ... [p] sent2,1 [s] sent2,2 [s] ...” 
where [s] and [p] are special tokens representing sentences and paragraphs. The special tokens were added to the RoBERTa vocabulary and randomly initialized before task finetuning.

In [1]:
# helper functions to convert hotpotqa to squard format modified from  https://github.com/chiayewken/bert-qa/blob/master/run_hotpot.py

import tqdm

def create_example_dict(context, answers, id, is_impossible, question, is_sup_fact, is_supporting_para):
    return {
        "context": context,
        "qas": [                        # each context corresponds to only one qa in hotpotqa
            {
                "answers": answers,
                "id": id,
                "is_impossible": is_impossible,
                "question": question,
                "is_sup_fact": is_sup_fact,
                "is_supporting_para": is_supporting_para
            }
        ],
    }


def create_para_dict(example_dicts):
    if type(example_dicts) == dict:
        example_dicts = [example_dicts]   # each paragraph corresponds to only one [context, qas] in hotpotqa
    return {"paragraphs": example_dicts}   

In [2]:
import re
def convert_hotpot_to_squad_format(json_dict, gold_paras_only=False):
    
    """function to convert hotpotqa to squard format.


    Note: A context corresponds to several qas in SQuard. In hotpotqa, one question corresponds to several paragraphs as context. 
          "paragraphs" means different: each paragraph in SQuard contains a context and a list of qas; while 10 paragraphs in hotpotqa concatenated into a context for one question.

    Args:
        json_dict: The original data load from hotpotqa file.
        gold_paras_only: when is true, only use the 2 paragraphs that contain the gold supporting facts; if false, use all the 10 paragraphs
 

    Returns:
        new_dict: The converted dict of hotpotqa dataset, use it as a dict would load from SQuAD json file
                  usage: input_data = new_dict["data"]   https://github.com/google-research/bert/blob/eedf5716ce1268e56f0a50264a88cafad334ac61/run_squad.py#L230

    """

    new_dict = {"data": []} 
    for example in json_dict: 

        support_para = set(
            para_title for para_title, _ in example["supporting_facts"]
        )
        sp_set = set(list(map(tuple, example['supporting_facts'])))
        
        raw_contexts = example["context"]
        if gold_paras_only: 
            raw_contexts = [lst for lst in raw_contexts if lst[0] in support_para]
            
        contexts = [" <s>".join(lst[1]) for lst in raw_contexts]
        context = "<p> " + " <p> ".join(contexts)
        
        is_supporting_para = []  # a boolean list with 10 True/False elements, one for each paragraph
        is_sup_fact = []         # a boolean list with True/False elements, one for each context sentence
        for para_title, para_lines in raw_contexts:
            is_supporting_para.append(para_title in support_para)   
            for sent_id, sent in enumerate(para_lines):
                is_sup_fact.append( (para_title, sent_id) in sp_set )


        answer = example["answer"].strip() 
        if answer.lower() == 'yes':
            answers = [{"answer_start": -1, "answer_end": -1, "text": answer}] 
        elif answer.lower() == 'no':
            answers = [{"answer_start": -2, "answer_end": -2, "text": answer}] 
        else:
            answers = []          # keep all the occurences of answer in the context
            for m in re.finditer(re.escape(answer), context):    
                answer_start, answer_end = m.span() 
                answers.append({"answer_start": answer_start, "answer_end": answer_end, "text": answer})
             

        new_dict["data"].append(
            create_para_dict(
                create_example_dict(
                    context=context,
                    answers=answers,
                    id = example["_id"],
                    is_impossible=(answers == []),
                    question=example["question"],
                    is_sup_fact = is_sup_fact,
                    is_supporting_para = is_supporting_para 
                )
            )
        ) 
    return new_dict

In [3]:
# debug: check whether convert_hotpot_to_squad_format() works
import os
os.chdir('/xdisk/msurdeanu/fanluo/hotpotQA/')
#!cat /xdisk/msurdeanu/fanluo/hotpotQA/hotpot_train_v1.1.json | ../jq-linux64 -c '.[0:16]' > small.json

import json
with open("small.json", "r", encoding='utf-8') as f:  
    json_dict = convert_hotpot_to_squad_format(json.load(f))['data']
    print(json.dumps(json_dict[0], indent=2))

{
  "paragraphs": [
    {
      "context": "<p> Radio City is India's first private FM radio station and was started on 3 July 2001. <s> It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003). <s> It plays Hindi, English and regional songs. <s> It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007. <s> Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features. <s> The Radio station currently plays a mix of Hindi and Regional music. <s> Abraham Thomas is the CEO of the company. <p> Football in Albania existed before the Albanian Football Federation (FSHF) was created. <s> This was evidenced by the team's registration at the Balkan Cup tournament during 1929-1931, which started in 1929 (al

### longfomer's fine-tuning


- For answer span extraction we use BERT’s QA model with addition of a question type (yes/no/span) classification head over the first special token ([CLS]).

- For evidence extraction we apply 2 layer feedforward networks on top of the representations corresponding to sentence and paragraph tokens to get the corresponding evidence prediction scores and use binary cross entropy loss to train the model.

- We combine span, question classification, sentence, and paragraphs losses and train the model in a multitask way using linear combination of losses.


In [5]:
### Section2: This is modified from longfomer's fine-tuning with triviaqa.py from https://github.com/allenai/longformer/blob/master/scripts/triviaqa.py
# !conda install transformers --yes
# !conda install cudatoolkit=10.0 --yes
# !python -m pip install git+https://github.com/allenai/longformer.git
# !conda install -c conda-forge regex --force-reinstall --yes
# !conda install pytorch-lightning -c conda-forge
# !pip install jdc 
# !pip install test-tube 
# !conda install ipywidgets --yes
# !conda update --force conda --yes  
# !jupyter nbextension enable --py widgetsnbextension 
# !conda install jupyter --yes

# need to run this every time start this notebook, to add python3.7/site-packages to sys.pat, in order to import ipywidgets, which is used when RobertaTokenizer.from_pretrained('roberta-base') 
import sys
sys.path.insert(-1, '/xdisk/msurdeanu/fanluo/miniconda3/lib/python3.7/site-packages')

import os
from collections import defaultdict
import json
import string
import random
import numpy as np
import torch
from torch.optim.lr_scheduler import LambdaLR

from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer 

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.overrides.data_parallel import LightningDistributedDataParallel
from pytorch_lightning.logging import TestTubeLogger    # sometimes pytorch_lightning.loggers works instead


from longformer.longformer import Longformer
from longformer.sliding_chunks import pad_to_window_size
import jdc


#### class hotpotqaDataset

##### \_\_init\_\_, \_\_getitem\_\_ and \_\_len\_\_ 

In [6]:
class hotpotqaDataset(Dataset):
    """
    Largely based on
    https://github.com/allenai/allennlp/blob/master/allennlp/data/dataset_readers/reading_comprehension/triviaqa.py
    and
    https://github.com/huggingface/transformers/blob/master/examples/run_squad.py
    """
    
    
    def __init__(self, file_path, tokenizer, max_seq_len, max_doc_len, doc_stride,
                 max_num_answers, ignore_seq_with_no_answers, max_question_len):
        assert os.path.isfile(file_path)
        self.file_path = file_path
        with open(self.file_path, "r", encoding='utf-8') as f:
            print(f'reading file: {self.file_path}')
            self.data_json = convert_hotpot_to_squad_format(json.load(f))['data']
#             print(self.data_json[0])
        
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.max_doc_len = max_doc_len
        self.doc_stride = doc_stride
        self.max_num_answers = max_num_answers
        self.ignore_seq_with_no_answers = ignore_seq_with_no_answers
        self.max_question_len = max_question_len

        print(tokenizer.all_special_tokens)
        print(tokenizer.all_special_ids)
    
        # A mapping from qid to an int, which can be synched across gpus using `torch.distributed`
        if 'train' not in self.file_path:  # only for the evaluation set 
            self.val_qid_string_to_int_map =  \
                {
                    entry["paragraphs"][0]['qas'][0]['id']: index
                    for index, entry in enumerate(self.data_json)
                }
        else:
            self.val_qid_string_to_int_map = None
            
            
    def __len__(self):
        return len(self.data_json)

    def __getitem__(self, idx):
        entry = self.data_json[idx]
        tensors_list = self.one_example_to_tensors(entry, idx)
        assert len(tensors_list) == 1
        return tensors_list[0]
    

##### one_example_to_tensors

In [7]:
    %%add_to hotpotqaDataset
    def one_example_to_tensors(self, example, idx):
        def is_whitespace(c):
            if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
                return True
            return False
        tensors_list = []
        for paragraph in example["paragraphs"]:  # example["paragraphs"] only contains one paragraph in hotpotqa
            paragraph_text = paragraph["context"]
            doc_tokens = []
            char_to_word_offset = []
            prev_is_whitespace = True
            for c in paragraph_text:
                if is_whitespace(c):
                    prev_is_whitespace = True
                else:
                    if prev_is_whitespace:
                        doc_tokens.append(c) # add a new token
                    else:
                        doc_tokens[-1] += c  # append the character to the last token
                    prev_is_whitespace = False
                char_to_word_offset.append(len(doc_tokens) - 1)

            for qa in paragraph["qas"]:
                question_text = qa["question"]
                start_position = None
                end_position = None
                orig_answer_text = None
                
                # keep all answers in the document, not just the first matched answer. It also added the list of textual answers to make evaluation easy.
                answer_spans = []
                for answer in qa["answers"]:
                    orig_answer_text = answer["text"]
                    answer_start = answer["answer_start"]
                    answer_end = answer["answer_end"]  
                    if(answer_start >= 0 and answer_end > 0):
                        try:
                            start_word_position = char_to_word_offset[answer_start]
                            end_word_position = char_to_word_offset[answer_end-1]
                        except:
                            print(f'Reading example {idx} failed')
                            start_word_position = -3
                            end_word_position = -3
                    else:
                        start_word_position = answer["answer_start"]
                        end_word_position = answer["answer_end"]
                    answer_spans.append({'start': start_word_position, 'end': end_word_position})

                # ===== Given an example, convert it into tensors  =============
                query_tokens = self.tokenizer.tokenize(question_text)
                query_tokens = query_tokens[:self.max_question_len]
                tok_to_orig_index = []
                orig_to_tok_index = []
                all_doc_tokens = []
                for (i, token) in enumerate(doc_tokens):
                    orig_to_tok_index.append(len(all_doc_tokens))
                    # hack: the line below should have been `self.tokenizer.tokenize(token')`
                    # but roberta tokenizer uses a different subword if the token is the beginning of the string
                    # or in the middle. So for all tokens other than the first, simulate that it is not the first
                    # token by prepending a period before tokenizing, then dropping the period afterwards
                    sub_tokens = self.tokenizer.tokenize(f'. {token}')[1:] if i > 0 else self.tokenizer.tokenize(token)
                    for sub_token in sub_tokens:
                        tok_to_orig_index.append(i)
                        all_doc_tokens.append(sub_token)

                all_doc_tokens = all_doc_tokens[:self.max_doc_len]

                # The -4 accounts for [CLS], [q], [/q] and [/s]
                max_tokens_per_doc_slice = self.max_seq_len - len(query_tokens) - 4
                assert max_tokens_per_doc_slice > 0
                if self.doc_stride < 0:                           # default
                    # negative doc_stride indicates no sliding window, but using first slice
                    self.doc_stride = -100 * len(all_doc_tokens)  # large -ve value for the next loop to execute once
                input_ids_list = []
                input_mask_list = []
                segment_ids_list = []
                start_positions_list = []
                end_positions_list = []
                q_type_list = []
                for slice_start in range(0, len(all_doc_tokens), max_tokens_per_doc_slice - self.doc_stride):    # execute once by default
                    slice_end = min(slice_start + max_tokens_per_doc_slice, len(all_doc_tokens))

                    doc_slice_tokens = all_doc_tokens[slice_start:slice_end]
                    tokens = [self.tokenizer.cls_token] + ["<q>"] + query_tokens + ["</q>"] + doc_slice_tokens + [self.tokenizer.eos_token] 
                    segment_ids = [0] * (len(query_tokens) + 3) + [1] * (len(doc_slice_tokens) + 1)
                    assert len(segment_ids) == len(tokens)

                    input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
                    input_mask = [1] * len(input_ids)

                    if self.doc_stride >= 0:  # no need to pad if document is not strided
                        # Zero-pad up to the sequence length.
                        padding_len = self.max_seq_len - len(input_ids)
                        input_ids.extend([self.tokenizer.pad_token_id] * padding_len)
                        input_mask.extend([0] * padding_len)
                        segment_ids.extend([0] * padding_len)

                        assert len(input_ids) == self.max_seq_len
                        assert len(input_mask) == self.max_seq_len
                        assert len(segment_ids) == self.max_seq_len

                    doc_offset = len(query_tokens) + 3 - slice_start
                    start_positions = []
                    end_positions = []
                    q_type = []
                    for answer_span in answer_spans:
                        start_position = answer_span['start']
                        end_position = answer_span['end']
                        if(start_position >= 0):
                            tok_start_position_in_doc = orig_to_tok_index[start_position]
                            not_end_of_doc = int(end_position + 1 < len(orig_to_tok_index))
                            tok_end_position_in_doc = orig_to_tok_index[end_position + not_end_of_doc] - not_end_of_doc
                            if tok_start_position_in_doc < slice_start or tok_end_position_in_doc > slice_end:
                                # this answer is outside the current slice
                                continue
                            start_positions.append(tok_start_position_in_doc + doc_offset)
                            end_positions.append(tok_end_position_in_doc + doc_offset)
                            q_type.append(0)
                        elif(start_position == -1):
                            q_type.append(1)
                            start_positions.append(-1)  # will be ignored
                            end_positions.append(-1)     
                        elif(start_position == -2):
                            q_type.append(2)
                            start_positions.append(-1)
                            end_positions.append(-1)     
                        else:
                            continue
                    assert len(start_positions) == len(end_positions)
                    if self.ignore_seq_with_no_answers and len(start_positions) == 0:
                        continue

                    # answers from start_positions and end_positions if > self.max_num_answers
                    start_positions = start_positions[:self.max_num_answers]
                    end_positions = end_positions[:self.max_num_answers]

                    # -1 padding up to self.max_num_answers
                    padding_len = self.max_num_answers - len(start_positions)
                    start_positions.extend([-1] * padding_len)
                    end_positions.extend([-1] * padding_len)

                    # replace duplicate start/end positions with `-1` because duplicates can result into -ve loss values
                    found_start_positions = set()
                    found_end_positions = set()
                    for i, (start_position, end_position) in enumerate(zip(start_positions, end_positions)):
                        if start_position in found_start_positions:
                            start_positions[i] = -1
                        if end_position in found_end_positions:
                            end_positions[i] = -1
                        found_start_positions.add(start_position)
                        found_end_positions.add(end_position)

                    input_ids_list.append(input_ids)
                    input_mask_list.append(input_mask)
                    segment_ids_list.append(segment_ids)
                    start_positions_list.append(start_positions)
                    end_positions_list.append(end_positions)
                    q_type_list.append(q_type)
                tensors_list.append((torch.tensor(input_ids_list), torch.tensor(input_mask_list),
                                     torch.tensor(segment_ids_list),
                                     torch.tensor(start_positions_list), torch.tensor(end_positions_list), torch.tensor(q_type_list),
                                     qa['id']))   
#                 tensors_list.append(all_doc_tokens)
        return tensors_list

##### collate_one_doc_and_lists

In [8]:
    %%add_to hotpotqaDataset
    @staticmethod
    def collate_one_doc_and_lists(batch):
        num_metadata_fields = 1  # qids  
        fields = [x for x in zip(*batch)]
        stacked_fields = [torch.stack(field) for field in fields[:-num_metadata_fields]]  # don't stack metadata fields
        stacked_fields.extend(fields[-num_metadata_fields:])  # add them as lists not torch tensors

        # always use batch_size=1 where each batch is one document
        # will use grad_accum to increase effective batch size
        assert len(batch) == 1
        fields_with_batch_size_one = [f[0] for f in stacked_fields]
        return fields_with_batch_size_one

##### class info

In [13]:
dir(hotpotqaDataset)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'collate_one_doc_and_lists',
 'one_example_to_tensors']

In [37]:
from inspect import getmembers
getmembers(hotpotqaDataset)

[('__add__', <function torch.utils.data.dataset.Dataset.__add__(self, other)>),
 ('__class__', type),
 ('__delattr__', <slot wrapper '__delattr__' of 'object' objects>),
 ('__dict__',
  mappingproxy({'__module__': '__main__',
                '__doc__': '\n    Largely based on\n    https://github.com/allenai/allennlp/blob/master/allennlp/data/dataset_readers/reading_comprehension/triviaqa.py\n    and\n    https://github.com/huggingface/transformers/blob/master/examples/run_squad.py\n    ',
                '__init__': <function __main__.hotpotqaDataset.__init__(self, file_path, tokenizer, max_seq_len, max_doc_len, doc_stride, max_num_answers, ignore_seq_with_no_answers, max_question_len)>,
                '__len__': <function __main__.hotpotqaDataset.__len__(self)>,
                '__getitem__': <function __main__.hotpotqaDataset.__getitem__(self, idx)>,
                'one_example_to_tensors': <function __main__.one_example_to_tensors(self, example, idx)>,
                'collate_one

In [38]:
from inspect import isfunction
functions_list = [o for o in getmembers(hotpotqaDataset) if isfunction(o[1])]
functions_list

[('__add__', <function torch.utils.data.dataset.Dataset.__add__(self, other)>),
 ('__getitem__', <function __main__.hotpotqaDataset.__getitem__(self, idx)>),
 ('__init__',
  <function __main__.hotpotqaDataset.__init__(self, file_path, tokenizer, max_seq_len, max_doc_len, doc_stride, max_num_answers, ignore_seq_with_no_answers, max_question_len)>),
 ('__len__', <function __main__.hotpotqaDataset.__len__(self)>),
 ('collate_one_doc_and_lists',
  <function __main__.collate_one_doc_and_lists(batch)>),
 ('one_example_to_tensors',
  <function __main__.one_example_to_tensors(self, example, idx)>)]

In [39]:
import inspect
inspect.getmro(hotpotqaDataset)  # a hierarchy of classes 

(__main__.hotpotqaDataset, torch.utils.data.dataset.Dataset, object)

In [56]:
inspect.getfullargspec(hotpotqaDataset.one_example_to_tensors)

FullArgSpec(args=['self', 'example', 'idx'], varargs=None, varkw=None, defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [40]:
help(hotpotqaDataset)

Help on class hotpotqaDataset in module __main__:

class hotpotqaDataset(torch.utils.data.dataset.Dataset)
 |  hotpotqaDataset(file_path, tokenizer, max_seq_len, max_doc_len, doc_stride, max_num_answers, ignore_seq_with_no_answers, max_question_len)
 |  
 |  Largely based on
 |  https://github.com/allenai/allennlp/blob/master/allennlp/data/dataset_readers/reading_comprehension/triviaqa.py
 |  and
 |  https://github.com/huggingface/transformers/blob/master/examples/run_squad.py
 |  
 |  Method resolution order:
 |      hotpotqaDataset
 |      torch.utils.data.dataset.Dataset
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, idx)
 |  
 |  __init__(self, file_path, tokenizer, max_seq_len, max_doc_len, doc_stride, max_num_answers, ignore_seq_with_no_answers, max_question_len)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __len__(self)
 |  
 |  one_example_to_tensors(self, example, idx)
 |  
 |  ------------------------------

#### class hotpotqa

##### \_\_init\_\_ and forward

In [9]:
class hotpotqa(pl.LightningModule):
    def __init__(self, args):
        super(hotpotqa, self).__init__()
        self.args = args
        self.hparams = args

        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        num_new_tokens = self.tokenizer.add_special_tokens({"additional_special_tokens": ["<p>", "<q>", "<\q>"]})
        print(self.tokenizer.all_special_tokens)
        self.tokenizer.model_max_length = self.args.max_seq_len
        self.model = self.load_model()
        self.model.resize_token_embeddings(len(self.tokenizer))
        self.num_labels = 2
        self.qa_outputs = torch.nn.Linear(self.model.config.hidden_size, self.num_labels)
        self.linear_type = torch.nn.Linear(self.model.config.hidden_size, 3)  #  question type (yes/no/span) classification 
        self.train_dataloader_object = self.val_dataloader_object = self.test_dataloader_object = None
    
    def load_model(self):
#         model = Longformer.from_pretrained(self.args.model_path)
        model = Longformer.from_pretrained('longformer-base-4096')
        for layer in model.encoder.layer:
            layer.attention.self.attention_mode = self.args.attention_mode
            self.args.attention_window = layer.attention.self.attention_window

        print("Loaded model with config:")
        print(model.config)

        for p in model.parameters():
            p.requires_grad_(True)
        model.train()
        return model
 
    def forward(self, input_ids, attention_mask, segment_ids, start_positions, end_positions, q_types):
        question_end_index = self._get_question_end_index(input_ids)
        # Each batch is one document, and each row of the batch is a chunck of the document.
        # Make sure all rows have the same question length.
        assert (question_end_index[0].float() == question_end_index.float().mean()).item()

        # local attention everywhere
        attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)
        # global attention for the question tokens
        attention_mask[:, :question_end_index.item()] = 2

        # sliding_chunks implemenation of selfattention requires that seqlen is multiple of window size
        input_ids, attention_mask = pad_to_window_size(
            input_ids, attention_mask, self.args.attention_window, self.tokenizer.pad_token_id)

        sequence_output = self.model(
                input_ids,
                attention_mask=attention_mask)[0]

        # The pretrained hotpotqa model wasn't trained with padding, so remove padding tokens
        # before computing loss and decoding.
        padding_len = input_ids[0].eq(self.tokenizer.pad_token_id).sum()
        if padding_len > 0:
            sequence_output = sequence_output[:, :-padding_len]

        predict_type = self.linear_type(sequence_output[:,0])

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        outputs = (start_logits, end_logits,)
        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)

            if not self.args.regular_softmax_loss:
                # loss function suggested in section 2.2 here https://arxiv.org/pdf/1710.10723.pdf
                # NOTE: this returns sum of losses, not mean, so loss won't be normalized across different batch sizes
                # but batch size is always 1, so this is not a problem
                start_loss = self.or_softmax_cross_entropy_loss_one_doc(start_logits, start_positions, ignore_index=-1)
                end_loss = self.or_softmax_cross_entropy_loss_one_doc(end_logits, end_positions, ignore_index=-1)
            else:
                loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-1)
                start_positions = start_positions[:, 0:1]
                end_positions = end_positions[:, 0:1]
                start_loss = loss_fct(start_logits, start_positions[:, 0])
                end_loss = loss_fct(end_logits, end_positions[:, 0])

            total_loss = (start_loss + end_loss) / 2
            outputs = (total_loss,) + outputs

        return outputs  # (loss), start_logits, end_logits, (hidden_states), (attentions)

    def or_softmax_cross_entropy_loss_one_doc(self, logits, target, ignore_index=-1, dim=-1):
        """loss function suggested in section 2.2 here https://arxiv.org/pdf/1710.10723.pdf"""
        assert logits.ndim == 2
        assert target.ndim == 2
        assert logits.size(0) == target.size(0)

        # with regular CrossEntropyLoss, the numerator is only one of the logits specified by the target
        # here, the numerator is the sum of a few potential targets, where some of them is the correct answer

        # compute a target mask
        target_mask = target == ignore_index
        # replaces ignore_index with 0, so `gather` will select logit at index 0 for the msked targets
        masked_target = target * (1 - target_mask.long())
        # gather logits
        gathered_logits = logits.gather(dim=dim, index=masked_target)
        # Apply the mask to gathered_logits. Use a mask of -inf because exp(-inf) = 0
        gathered_logits[target_mask] = float('-inf')

        # each batch is one example
        gathered_logits = gathered_logits.view(1, -1)
        logits = logits.view(1, -1)

        # numerator = log(sum(exp(gathered logits)))
        log_score = torch.logsumexp(gathered_logits, dim=dim, keepdim=False)
        # denominator = log(sum(exp(logits)))
        log_norm = torch.logsumexp(logits, dim=dim, keepdim=False)

        # compute the loss
        loss = -(log_score - log_norm)

        # some of the examples might have a loss of `inf` when `target` is all `ignore_index`.
        # remove those from the loss before computing the sum. Use sum instead of mean because
        # it is easier to compute
        return loss[~torch.isinf(loss)].sum()


##### **dataloader**

In [9]:
# debug: check loaded dataset by DataLoader
 
dataset = hotpotqaDataset(file_path= args.train_dataset, tokenizer=RobertaTokenizer.from_pretrained('roberta-base'),
                          max_seq_len= args.max_seq_len, max_doc_len= args.max_doc_len,
                          doc_stride= args.doc_stride,
                          max_num_answers= args.max_num_answers,
                          max_question_len= args.max_question_len,
                          ignore_seq_with_no_answers= args.ignore_seq_with_no_answers)
print(len(dataset))

dl = DataLoader(dataset, batch_size=1, shuffle=None,
                    num_workers=args.num_workers, sampler=None,
                    collate_fn=hotpotqaDataset.collate_one_doc_and_lists)

example = dataset[0] 
print(example)
# [input_ids, input_mask, segment_ids, subword_starts, subword_ends, _, qids] = example1


# print(input_ids)
# print(input_mask) 
# print(segment_ids) 
# print(subword_starts) 
# print(subword_ends) 
# print(qids)


NameError: name 'args' is not defined

In [10]:
%%add_to hotpotqa
@pl.data_loader
def train_dataloader(self):
    if self.train_dataloader_object is not None:
        return self.train_dataloader_object
    dataset = hotpotqaDataset(file_path=self.args.train_dataset, tokenizer=self.tokenizer,
                              max_seq_len=self.args.max_seq_len, max_doc_len=self.args.max_doc_len,
                              doc_stride=self.args.doc_stride,
                              max_num_answers=self.args.max_num_answers,
                              max_question_len=self.args.max_question_len,
                              ignore_seq_with_no_answers=self.args.ignore_seq_with_no_answers)

    sampler = torch.utils.data.distributed.DistributedSampler(dataset) if self.trainer.use_ddp else None
    dl = DataLoader(dataset, batch_size=1, shuffle=(sampler is None),
                    num_workers=self.args.num_workers, sampler=sampler,
                    collate_fn=hotpotqaDataset.collate_one_doc_and_lists)

    self.train_dataloader_object = dl
    return self.train_dataloader_object

@pl.data_loader
def val_dataloader(self):
    if self.val_dataloader_object is not None:
        return self.val_dataloader_object
    dataset = hotpotqaDataset(file_path=self.args.dev_dataset, tokenizer=self.tokenizer,
                              max_seq_len=self.args.max_seq_len, max_doc_len=self.args.max_doc_len,
                              doc_stride=self.args.doc_stride,
                              max_num_answers=self.args.max_num_answers,
                              max_question_len=self.args.max_question_len,
                              ignore_seq_with_no_answers=False)  # evaluation data should keep all examples
    sampler = torch.utils.data.distributed.DistributedSampler(dataset) if self.trainer.use_ddp else None
    dl = DataLoader(dataset, batch_size=1, shuffle=(sampler is None),
                    num_workers=self.args.num_workers, sampler=sampler,
                    collate_fn=hotpotqaDataset.collate_one_doc_and_lists)
    self.val_dataloader_object = dl
    return self.val_dataloader_object

@pl.data_loader
def test_dataloader(self):
    if self.test_dataloader_object is not None:
        return self.test_dataloader_object
    dataset = hotpotqaDataset(file_path=self.args.dev_dataset, tokenizer=self.tokenizer,
                              max_seq_len=self.args.max_seq_len, max_doc_len=self.args.max_doc_len,
                              doc_stride=self.args.doc_stride,
                              max_num_answers=self.args.max_num_answers,
                              max_question_len=self.args.max_question_len,
                              ignore_seq_with_no_answers=False)  # evaluation data should keep all examples

    dl = DataLoader(dataset, batch_size=1, shuffle=False,
                    num_workers=self.args.num_workers, sampler=None,
                    collate_fn=hotpotqaDataset.collate_one_doc_and_lists)
    self.test_dataloader_object = dl
    return self.test_dataloader_object

##### configure_ddp

In [10]:
 %%add_to hotpotqa
 # A hook to overwrite to define your own DDP(DistributedDataParallel) implementation init. 
 # The only requirement is that: 
 # 1. On a validation batch the call goes to model.validation_step.
 # 2. On a training batch the call goes to model.training_step.
 # 3. On a testing batch, the call goes to model.test_step
 def configure_ddp(self, model, device_ids):
    model = LightningDistributedDataParallel(
        model,
        device_ids=device_ids,
        find_unused_parameters=True
    )
    return model

##### **configure_optimizers**

In [11]:
%%add_to hotpotqa
def configure_optimizers(self):
    # Set up optimizers and (optionally) learning rate schedulers
    optimizer = torch.optim.Adam(self.parameters(), lr=self.args.lr)

    def lr_lambda(current_step):
        if current_step < self.args.warmup:
            return float(current_step) / float(max(1, self.args.warmup))
        return max(0.0, float(self.args.steps - current_step) / float(max(1, self.args.steps - self.args.warmup)))
    self.scheduler = LambdaLR(optimizer, lr_lambda, last_epoch=-1)  # scheduler is not saved in the checkpoint, but global_step is, which is enough to restart
    self.scheduler.step(self.global_step)

    return optimizer
    

##### optimizer_step

In [12]:
%%add_to hotpotqa
# A hook to do a lot of non-standard training tricks such as learning-rate warm-up
def optimizer_step(self, current_epoch, batch_nb, optimizer, optimizer_i, second_order_closure=None):
    optimizer.step()
    optimizer.zero_grad()
    self.scheduler.step(self.global_step)

##### **training_step**

In [13]:
%%add_to hotpotqa
def training_step(self, batch, batch_nb):
    # do the forward pass and calculate the loss for a batch
    input_ids, input_mask, segment_ids, subword_starts, subword_ends, qids = batch
    output = self.forward(input_ids, input_mask, segment_ids, subword_starts, subword_ends)
    loss = output[0]
    lr = loss.new_zeros(1) + self.trainer.optimizers[0].param_groups[0]['lr']
    tensorboard_logs = {'train_loss': loss, 'lr': lr,
                        'input_size': input_ids.numel(),
                        'mem': torch.cuda.memory_allocated(input_ids.device) / 1024 ** 3}
    return {'loss': loss, 'log': tensorboard_logs}
from functools import wraps 

##### validation_step

In [55]:
%%add_to hotpotqa
# When the validation_step is called, the model has been put in eval mode and PyTorch gradients have been disabled. At the end of validation, model goes back to training mode and gradients are enabled.
def validation_step(self, batch, batch_nb):
    input_ids, input_mask, segment_ids, subword_starts, subword_ends, qids = batch
    output = self.forward(input_ids, input_mask, segment_ids, subword_starts, subword_ends)
    loss, start_logits, end_logits = output[:3]
    answers = self.decode(input_ids, start_logits, end_logits)

    # each batch is one document
    answers = sorted(answers, key=lambda x: x['score'], reverse=True)[0:1]
    qids = [qids] 

    f1_scores = [evaluation_utils.metric_max_over_ground_truths(evaluation_utils.f1_score, answer['text'])
                 for answer in answers]
    # TODO: if slow, skip em_scores, and use (f1_score == 1.0) instead
    em_scores = [evaluation_utils.metric_max_over_ground_truths(evaluation_utils.exact_match_score, answer['text'])
                 for answer in answers]
    answer_scores = [answer['score'] for answer in answers]  # start_logit + end_logit
    assert len(answer_scores) == len(f1_scores) == len(em_scores) == len(qids) == 1

    # TODO: delete this metric
    pred_subword_starts = start_logits.argmax(dim=1)
    pred_subword_ends = end_logits.argmax(dim=1)
    exact_match = (subword_ends[:, 0].squeeze(dim=-1) == pred_subword_ends).float() *  \
                  (subword_starts[:, 0].squeeze(dim=-1) == pred_subword_starts).float()

    return {'vloss': loss, 'vem': exact_match.mean(),
            'qids': qids, 'answer_scores': answer_scores,
            'f1': f1_scores, 'em': em_scores}

##### validation_end

In [56]:
%%add_to hotpotqa
# If you didn't define a validation_step, this won't be called. Called at the end of the validation loop with the outputs of validation_step.
def validation_end(self, outputs):
    avg_loss = torch.stack([x['vloss'] for x in outputs]).mean()
    avg_em = torch.stack([x['vem'] for x in outputs]).mean()
    string_qids = [item for sublist in outputs for item in sublist['qids']]
    int_qids = [self.val_dataloader_object.dataset.val_qid_string_to_int_map[qid] for qid in string_qids]
    answer_scores = [item for sublist in outputs for item in sublist['answer_scores']]
    f1_scores = [item for sublist in outputs for item in sublist['f1']]
    em_scores = [item for sublist in outputs for item in sublist['em']]
    print(f'before sync --> sizes: {len(int_qids)}, {len(answer_scores)}, {len(f1_scores)}, {len(em_scores)}')
    if self.trainer.use_ddp:
        torch.distributed.all_reduce(avg_loss, op=torch.distributed.ReduceOp.SUM)
        avg_loss /= self.trainer.world_size
        torch.distributed.all_reduce(avg_em, op=torch.distributed.ReduceOp.SUM)
        avg_em /= self.trainer.world_size

        int_qids = self.sync_list_across_gpus(int_qids, avg_loss.device, torch.int)
        answer_scores = self.sync_list_across_gpus(answer_scores, avg_loss.device, torch.float)
        f1_scores = self.sync_list_across_gpus(f1_scores, avg_loss.device, torch.float)
        em_scores = self.sync_list_across_gpus(em_scores, avg_loss.device, torch.int)
    print(f'after sync --> sizes: {len(int_qids)}, {len(answer_scores)}, {len(f1_scores)}, {len(em_scores)}')

    # Because of having multiple documents per questions, some questions might have multiple corresponding answers
    # Here, we only keep the answer with the highest answer_score
    qa_with_duplicates = defaultdict(list)
    for qid, answer_score, f1_score, em_score in zip(int_qids, answer_scores, f1_scores, em_scores):
        qa_with_duplicates[qid].append({'answer_score': answer_score, 'f1': f1_score, 'em': em_score})
    f1_scores = []
    em_scores = []
    for qid, answer_metrics in qa_with_duplicates.items():
        top_answer = sorted(answer_metrics, key=lambda x: x['answer_score'], reverse=True)[0]
        f1_scores.append(top_answer['f1'])
        em_scores.append(top_answer['em'])
    avg_val_f1 = sum(f1_scores) / len(f1_scores)
    avg_val_em = sum(em_scores) / len(em_scores)

    logs = {'val_loss': avg_loss, 'val_em': avg_em, 'avg_val_f1': avg_val_f1, 'avg_val_em': avg_val_em}

    return {'avg_val_loss': avg_loss, 'log': logs, 'progress_bar': logs}

def sync_list_across_gpus(self, l, device, dtype):
    l_tensor = torch.tensor(l, device=device, dtype=dtype)
    gather_l_tensor = [torch.ones_like(l_tensor) for _ in range(self.trainer.world_size)]
    torch.distributed.all_gather(gather_l_tensor, l_tensor)
    return torch.cat(gather_l_tensor).tolist()


##### test_step

In [92]:
%%add_to hotpotqa
def test_step(self, batch, batch_nb):
    input_ids, input_mask, segment_ids, subword_starts, subword_ends, qids = batch
    output = self.forward(input_ids, input_mask, segment_ids, subword_starts, subword_ends)
    loss, start_logits, end_logits = output[:3]
    answers = self.decode(input_ids, start_logits, end_logits)

    # each batch is one document
    answers = sorted(answers, key=lambda x: x['score'], reverse=True)[0:1]
    qids = [qids]
    assert len(answers) == len(qids)
    return {'qids': qids, 'answers': answers}

def _get_question_end_index(self, input_ids):
    eos_token_indices = (input_ids == self.tokenizer.eos_token_id).nonzero()
    assert eos_token_indices.ndim == 2
    assert eos_token_indices.size(0) == 2 * input_ids.size(0)
    assert eos_token_indices.size(1) == 2
    return eos_token_indices.view(input_ids.size(0), 2, 2)[:, 0, 1]

def decode(self, input_ids, start_logits, end_logits):
    question_end_index = self._get_question_end_index(input_ids)

    # bsz x seqlen => bsz x n_best_size
    start_logits_indices = start_logits.topk(k=self.args.n_best_size, dim=-1).indices
    end_logits_indices = end_logits.topk(k=self.args.n_best_size, dim=-1).indices

    answers = []
    # This loop can't be vectorized, so loop over each example in the batch separetly
    for i in range(start_logits_indices.size(0)):  # bsz
        potential_answers = []
        for start_logit_index in start_logits_indices[i]:  # n_best_size
            for end_logit_index in end_logits_indices[i]:  # n_best_size
                if start_logit_index <= question_end_index[i]:
                    continue
                if end_logit_index <= question_end_index[i]:
                    continue
                if start_logit_index > end_logit_index:
                    continue
                answer_len = end_logit_index - start_logit_index + 1
                if answer_len > self.args.max_answer_length:
                    continue
                potential_answers.append({'start': start_logit_index, 'end': end_logit_index,
                                          'start_logit': start_logits[i][start_logit_index].item(),
                                          'end_logit': end_logits[i][end_logit_index].item()})
        sorted_answers = sorted(potential_answers, key=lambda x: (x['start_logit'] + x['end_logit']), reverse=True)
        if len(sorted_answers) == 0:
            answers.append({'text': 'NoAnswerFound', 'score': -1000000})
        else:
            answer = sorted_answers[0]
            answer_token_ids = input_ids[i, answer['start']: answer['end'] + 1]
            answer_tokens = self.tokenizer.convert_ids_to_tokens(answer_token_ids.tolist())
            text = self.tokenizer.convert_tokens_to_string(answer_tokens)
            score = answer['start_logit'] + answer['end_logit']
            answers.append({'text': text, 'score': score})
    return answers




##### test_end

In [93]:
%%add_to hotpotqa
def test_end(self, outputs):
    qids = [item for sublist in outputs for item in sublist['qids']]
    answers = [item for sublist in outputs for item in sublist['answers']]

    qa_with_duplicates = defaultdict(list)
    for qid, answer in zip(qids, answers):
        qa_with_duplicates[qid].append({'answer_score': answer['score'], 'answer_text': answer['text'], })

    qid_to_answer_text = {}
    for qid, answer_metrics in qa_with_duplicates.items():
        top_answer = sorted(answer_metrics, key=lambda x: x['answer_score'], reverse=True)[0]
        qid_to_answer_text[qid] = top_answer['answer_text']

    with open('predictions.json', 'w') as f:
        json.dump(qid_to_answer_text, f)

    return {'count': len(qid_to_answer_text)}

##### add_model_specific_args

In [14]:
%%add_to hotpotqa
@staticmethod
def add_model_specific_args(parser, root_dir):
    parser.add_argument("--save_dir", type=str, default='hotpotqa')
    parser.add_argument("--save_prefix", type=str, required=True)
    parser.add_argument("--train_dataset", type=str, required=False, help="Path to the training squad-format")
    parser.add_argument("--dev_dataset", type=str, required=True, help="Path to the dev squad-format")
    parser.add_argument("--batch_size", type=int, default=8, help="Batch size")
    parser.add_argument("--gpus", type=str, default='0',
                        help="Comma separated list of gpus. Default is gpu 0. To use CPU, use --gpus "" ")
    parser.add_argument("--warmup", type=int, default=200, help="Number of warmup steps")
    parser.add_argument("--lr", type=float, default=0.0001, help="Maximum learning rate")
    parser.add_argument("--val_every", type=float, default=0.2, help="Number of training steps between validations")
    parser.add_argument("--val_percent_check", default=1.00, type=float, help='Percent of validation data used')
    parser.add_argument("--num_workers", type=int, default=4, help="Number of data loader workers")
    parser.add_argument("--seed", type=int, default=1234, help="Seed")
    parser.add_argument("--epochs", type=int, default=30, help="Number of epochs")
    parser.add_argument("--max_seq_len", type=int, default=4096,
                        help="Maximum length of seq passed to the transformer model")
    parser.add_argument("--max_doc_len", type=int, default=4096,
                        help="Maximum number of wordpieces of the input document")
    parser.add_argument("--max_num_answers", type=int, default=64,
                        help="Maximum number of answer spans per document (64 => 94%)")
    parser.add_argument("--max_question_len", type=int, default=55,
                        help="Maximum length of the question")
    parser.add_argument("--doc_stride", type=int, default=-1,
                        help="Overlap between document chunks. Use -1 to only use the first chunk")
    parser.add_argument("--ignore_seq_with_no_answers", action='store_true',
                        help="each example should have at least one answer. Default is False")
    parser.add_argument("--disable_checkpointing", action='store_true', help="No logging or checkpointing")
    parser.add_argument("--n_best_size", type=int, default=20,
                        help="Number of answer candidates. Used at decoding time")
    parser.add_argument("--max_answer_length", type=int, default=30,
                        help="maximum num of wordpieces/answer. Used at decoding time")
    parser.add_argument("--regular_softmax_loss", action='store_true', help="IF true, use regular softmax. Default is using ORed softmax loss")
    parser.add_argument("--test", action='store_true', help="Test only, no training")
    parser.add_argument("--model_path", type=str, required=True,
                        help="Path to the checkpoint directory")
    parser.add_argument("--no_progress_bar", action='store_true', help="no progress bar. Good for printing")
    parser.add_argument("--attention_mode", type=str, choices=['tvm', 'sliding_chunks'],
                        default='sliding_chunks', help='Which implementation of selfattention to use')
    parser.add_argument("--fp32", action='store_true', help="default is fp16. Use --fp32 to switch to fp32")

    return parser

##### class info

In [124]:
dir(hotpotqa)

['__abstractmethods__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_apply',
 '_get_name',
 '_load_from_state_dict',
 '_named_members',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_version',
 'add_module',
 'apply',
 'backward',
 'bfloat16',
 'buffers',
 'children',
 'configure_apex',
 'configure_ddp',
 'configure_optimizers',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'freeze',
 'grad_norm',
 'half',
 'init_ddp_connection',
 'load_from_checkp

In [60]:
from inspect import getmembers, isfunction
getmembers(hotpotqa)

[('__abstractmethods__', frozenset({'configure_optimizers', 'training_step'})),
 ('__call__',
  <function torch.nn.modules.module.Module.__call__(self, *input, **kwargs)>),
 ('__class__', abc.ABCMeta),
 ('__delattr__',
  <function torch.nn.modules.module.Module.__delattr__(self, name)>),
 ('__dict__',
  mappingproxy({'__module__': '__main__',
                '__init__': <function __main__.hotpotqa.__init__(self, args)>,
                'load_model': <function __main__.hotpotqa.load_model(self)>,
                'forward': <function __main__.hotpotqa.forward(self, input_ids, attention_mask, segment_ids, start_positions, end_positions, q_types)>,
                'or_softmax_cross_entropy_loss_one_doc': <function __main__.hotpotqa.or_softmax_cross_entropy_loss_one_doc(self, logits, target, ignore_index=-1, dim=-1)>,
                '__doc__': None,
                '__abstractmethods__': frozenset({'configure_optimizers',
                           'training_step'}),
                '_abc_

In [62]:
functions_list = [o for o in getmembers(hotpotqa) if isfunction(o[1])]
functions_list

[('__call__',
  <function torch.nn.modules.module.Module.__call__(self, *input, **kwargs)>),
 ('__delattr__',
  <function torch.nn.modules.module.Module.__delattr__(self, name)>),
 ('__dir__', <function torch.nn.modules.module.Module.__dir__(self)>),
 ('__getattr__',
  <function torch.nn.modules.module.Module.__getattr__(self, name)>),
 ('__init__', <function __main__.hotpotqa.__init__(self, args)>),
 ('__repr__', <function torch.nn.modules.module.Module.__repr__(self)>),
 ('__setattr__',
  <function torch.nn.modules.module.Module.__setattr__(self, name, value)>),
 ('__setstate__',
  <function torch.nn.modules.module.Module.__setstate__(self, state)>),
 ('_apply', <function torch.nn.modules.module.Module._apply(self, fn)>),
 ('_get_name', <function torch.nn.modules.module.Module._get_name(self)>),
 ('_load_from_state_dict',
  <function torch.nn.modules.module.Module._load_from_state_dict(self, state_dict, prefix, local_metadata, strict, missing_keys, unexpected_keys, error_msgs)>),
 ('

In [65]:
import inspect
inspect.getmro(hotpotqa)  # a hierarchy of classes 

(__main__.hotpotqa,
 pytorch_lightning.core.lightning.LightningModule,
 abc.ABC,
 pytorch_lightning.core.grads.GradInformation,
 pytorch_lightning.core.saving.ModelIO,
 pytorch_lightning.core.hooks.ModelHooks,
 torch.nn.modules.module.Module,
 object)

In [26]:
help(hotpotqa)

Help on class hotpotqa in module __main__:

class hotpotqa(pytorch_lightning.core.lightning.LightningModule)
 |  hotpotqa(args)
 |  
 |  Helper class that provides a standard way to create an ABC using
 |  inheritance.
 |  
 |  Method resolution order:
 |      hotpotqa
 |      pytorch_lightning.core.lightning.LightningModule
 |      abc.ABC
 |      pytorch_lightning.core.grads.GradInformation
 |      pytorch_lightning.core.saving.ModelIO
 |      pytorch_lightning.core.hooks.ModelHooks
 |      torch.nn.modules.module.Module
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, args)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  forward(self, input_ids, attention_mask, segment_ids, start_positions, end_positions, q_types)
 |      Same as torch.nn.Module.forward(), however in Lightning you want this to define
 |      the  operations you want to use for prediction (ie: on a server or as a feature extractor).
 |      
 |      Normal

In [25]:
help(hotpotqa.configure_optimizers)

Help on function configure_optimizers in module pytorch_lightning.core.lightning:

configure_optimizers(self)
    This is where you choose what optimizers and learning-rate schedulers to use in your optimization.
    Normally you'd need one. But in the case of GANs or something more esoteric you might have multiple.
    
    Return: any of these 3 options:
        - Single optimizer
        - List or Tuple - List of optimizers
        - Two lists - The first list has multiple optimizers, the second a list of learning-rate schedulers
    
    Example
    -------
    
    .. code-block:: python
    
        # most cases
        def configure_optimizers(self):
            opt = Adam(self.parameters(), lr=0.01)
            return opt
    
        # multiple optimizer case (eg: GAN)
        def configure_optimizers(self):
            generator_opt = Adam(self.model_gen.parameters(), lr=0.01)
            disriminator_opt = Adam(self.model_disc.parameters(), lr=0.02)
            return genera

In [18]:
import inspect
code, line_no = inspect.getsourcelines(hotpotqa.training_step)
print(''.join(code))

def training_step(self, batch, batch_nb):
    # do the forward pass and calculate the loss for a batch
    input_ids, input_mask, segment_ids, subword_starts, subword_ends, qids = batch
    output = self.forward(input_ids, input_mask, segment_ids, subword_starts, subword_ends)
    loss = output[0]
    lr = loss.new_zeros(1) + self.trainer.optimizers[0].param_groups[0]['lr']
    tensorboard_logs = {'train_loss': loss, 'lr': lr,
                        'input_size': input_ids.numel(),
                        'mem': torch.cuda.memory_allocated(input_ids.device) / 1024 ** 3}
    return {'loss': loss, 'log': tensorboard_logs}



### main

In [17]:
def main(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed) 

In [25]:
    hotpotqa.__abstractmethods__=set()   # without this, got an error "Can't instantiate abstract class hotpotqa with abstract methods" if these two abstract methods are not implemented in the same cell where class hotpotqa defined 
    model = hotpotqa(args)

INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /home/u32/fanluo/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at /home/u32/fanluo/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:transformers.tokenization_utils_base:Assigning ['<p>', '<q>', '<\\q>'] to the additional_special_tokens key of the tokenizer
INFO:transformers.tokenization_utils:Adding <p> to the vocabulary
INFO:transformers.tokenization_utils:Adding <q> to the vocabulary
INFO:transformers.tokenization_utils:Adding <\q> to the vocabulary
INFO:transformers.configuration_uti

['<s>', '<s>', '<mask>', '<\\q>', '</s>', '</s>', '<q>', '<unk>', '<p>', '<pad>']


INFO:transformers.modeling_utils:All model checkpoint weights were used when initializing Longformer.

INFO:transformers.modeling_utils:All the weights of Longformer were initialized from the model checkpoint at longformer-base-4096.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use Longformer for predictions without further training.


Loaded model with config:
RobertaConfig {
  "attention_dilation": [
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1
  ],
  "attention_mode": "tvm",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    256,
    256,
    256,
    256,
    256,
    256,
    256,
    256,
    256,
    256,
    256,
    256
  ],
  "autoregressive": false,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 50265
}



In [ ]:
    logger = TestTubeLogger( # The TestTubeLogger adds a nicer folder structure to manage experiments and snapshots all hyperparameters you pass to a LightningModule.
        save_dir=args.save_dir,
        name=args.save_prefix,
        version=0  # always use version=0
    )

In [ ]:
    checkpoint_callback = ModelCheckpoint(
        filepath=os.path.join(args.save_dir, args.save_prefix, "checkpoints"),
        save_top_k=5,
        verbose=True,
        monitor='avg_val_f1',
        mode='max',
        prefix=''
    )

In [64]:
    args.gpus = [int(x) for x in args.gpus.split(',')] if args.gpus is not "" else None  # use CPU if no gpu provided
    train_set_size = 90447   # hardcode dataset size. Needed to compute number of steps for the lr scheduler
    num_devices = 1 or len(args.gpus)
    args.steps = args.epochs * train_set_size / (args.batch_size * num_devices)
    print(f'>>>>>>> #steps: {args.steps}, #epochs: {args.epochs}, batch_size: {args.batch_size * num_devices} <<<<<<<')

>>>>>>> #steps: 339176.25, #epochs: 30, batch_size: 8 <<<<<<<


In [73]:
    trainer = pl.Trainer(gpus=args.gpus, distributed_backend='ddp' if args.gpus and (len(args.gpus) > 1) else None,
                         track_grad_norm=-1, max_nb_epochs=args.epochs, early_stop_callback=None,
                         accumulate_grad_batches=args.batch_size,
                         val_check_interval=args.val_every,
                         val_percent_check=args.val_percent_check,
                         test_percent_check=args.val_percent_check,
                         logger=logger if not args.disable_checkpointing else False,
                         checkpoint_callback=checkpoint_callback if not args.disable_checkpointing else False,
                         show_progress_bar=not args.no_progress_bar,
                         use_amp=not args.fp32, amp_level='O2',
                         )
    

/Users/fan/Documents/Research/mul-hop-viz/experiments/hotpot/apex
Non-user install because site-packages writeable
Created temporary directory: /private/var/folders/js/1c5k58md2jddt8mvkw21l6f00000gn/T/pip-ephem-wheel-cache-im7m2m6s
Created temporary directory: /private/var/folders/js/1c5k58md2jddt8mvkw21l6f00000gn/T/pip-req-tracker-0sj48rpe
Initialized build tracking at /private/var/folders/js/1c5k58md2jddt8mvkw21l6f00000gn/T/pip-req-tracker-0sj48rpe
Created build tracker: /private/var/folders/js/1c5k58md2jddt8mvkw21l6f00000gn/T/pip-req-tracker-0sj48rpe
Entered build tracker: /private/var/folders/js/1c5k58md2jddt8mvkw21l6f00000gn/T/pip-req-tracker-0sj48rpe
Created temporary directory: /private/var/folders/js/1c5k58md2jddt8mvkw21l6f00000gn/T/pip-install-tgxhca21
Processing /Users/fan/Documents/Research/mul-hop-viz/experiments/hotpot/apex
  Created temporary directory: /private/var/folders/js/1c5k58md2jddt8mvkw21l6f00000gn/T/pip-req-build-ardufyha
  Added file:///Users/fan/Documents/Rese

  Found link https://files.pythonhosted.org/packages/11/31/c483614095176ddfa06ac99c2af4171375053b270842c7865ca0b4438dc1/pip-19.0.tar.gz#sha256=c82bf8bc00c5732f0dd49ac1dea79b6242a1bd42a5012e308ed4f04369b17e54 (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*), version: 19.0
  Found link https://files.pythonhosted.org/packages/46/dc/7fd5df840efb3e56c8b4f768793a237ec4ee59891959d6a215d63f727023/pip-19.0.1-py2.py3-none-any.whl#sha256=aae79c7afe895fb986ec751564f24d97df1331bb99cdfec6f70dada2f40c0044 (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*), version: 19.0.1
  Found link https://files.pythonhosted.org/packages/c8/89/ad7f27938e59db1f0f55ce214087460f65048626e2226531ba6cb6da15f0/pip-19.0.1.tar.gz#sha256=e81ddd35e361b630e94abeda4a1eddd36d47a90e71eb00f38f46b57f787cd1a5 (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*), version: 19.0.1
  Found link https://files.python

Removed file:///Users/fan/Documents/Research/mul-hop-viz/experiments/hotpot/apex from build tracker '/private/var/folders/js/1c5k58md2jddt8mvkw21l6f00000gn/T/pip-req-tracker-0sj48rpe'
Removed build tracker: '/private/var/folders/js/1c5k58md2jddt8mvkw21l6f00000gn/T/pip-req-tracker-0sj48rpe'


In [ ]:
    if not args.test:
        trainer.fit(model)
    trainer.test(model)

In [16]:
# debug: check args
import shlex
argString ='--train_dataset small.json --dev_dataset /Users/fan/Documents/Research/mul-hop-viz/experiments/hotpot/hotpot_dev_distractor_v1.json  \
    --gpus 0  --num_workers 4 \
    --max_seq_len 4096 --doc_stride -1  \
    --save_prefix hotpotqa-longformer  --model_path /Users/fan/Downloads/longformer-base-4096 --test'

import argparse
if __name__ == "__main__":
    main_arg_parser = argparse.ArgumentParser(description="hotpotqa")
    parser = hotpotqa.add_model_specific_args(main_arg_parser, os.getcwd())
    args = parser.parse_args(shlex.split(argString))
    print(args)
    main(args)

Namespace(attention_mode='sliding_chunks', batch_size=8, dev_dataset='/Users/fan/Documents/Research/mul-hop-viz/experiments/hotpot/hotpot_dev_distractor_v1.json', disable_checkpointing=False, doc_stride=-1, epochs=30, fp32=False, gpus='0', ignore_seq_with_no_answers=False, lr=0.0001, max_answer_length=30, max_doc_len=4096, max_num_answers=64, max_question_len=55, max_seq_len=4096, model_path='/Users/fan/Downloads/longformer-base-4096', n_best_size=20, no_progress_bar=False, num_workers=4, regular_softmax_loss=False, save_dir='hotpotqa', save_prefix='hotpotqa-longformer', seed=1234, test=True, train_dataset='small.json', val_every=0.2, val_percent_check=1.0, warmup=200)
